# View the Coadds

- author : Sylvie Dagoret-Campagne
- creation date :  2024-04-04
- update : 2024-04-08
- w_2024_10

### Schema for data
- https://dm.lsst.org/sdm_schemas/browser/imsim.html

In [ ]:
from lsst.daf.butler import Butler
import lsst
import pandas as pd
import numpy as np
repo = '/repo/embargo'
instrument = 'LSSTComCamSim'
where_clause = "instrument = \'" + f"{instrument}" +"\'"
skymap_name = "ops_rehersal_prep_2k_v1"
#collection = 'LSSTComCamSim/quickLook/24'
collection1 = 'LSSTComCamSim/runs/nightlyvalidation/20240402/d_2024_03_29/DM-43612'
collection2 = 'LSSTComCamSim/runs/nightlyvalidation/20240403/d_2024_03_29/DM-43612'
collection3 = 'LSSTComCamSim/runs/nightlyvalidation/20240404/d_2024_03_29/DM-43612'
#collection = 'LSSTComCamSim/runs/nightlyvalidation/20240404/d_2024_03_29/DM-43612'
collections = [collection1,collection2,collection3]
butler = Butler(repo, collections=collections)
registry = butler.registry

In [ ]:
import lsst.afw.image as afwImage
import lsst.afw.display as afwDisplay
import lsst.afw.table as afwTable
import lsst.geom as geom
import pprint
# Let us also set `lsst.afw.display` to use the `matplotlib` backend:
afwDisplay.setDefaultBackend('matplotlib')



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm 
import matplotlib.colors as colors
import matplotlib.cm as cmx
import matplotlib.dates as mdates
from matplotlib.colors import ListedColormap
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
from astropy.visualization import imshow_norm, MinMaxInterval, SqrtStretch
# Astropy
from astropy.visualization import ZScaleInterval, AsinhStretch

import seaborn as sns
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [ ]:
def get_corners_radec(wcs, bbox):
    """
    Return the corners in RA,Dec in degrees given the WCS and bounding box for an image.

    Parameters
    ----------
    wcs: image WCS returned by the Butler
    bbox: bounding box returned by the Butler

    Returns
    -------
    corners_ra, corners_dec in decimal degrees
    """

    corners_x = [bbox.beginX, bbox.beginX, bbox.endX, bbox.endX]
    corners_y = [bbox.beginY, bbox.endY, bbox.endY, bbox.beginY]
    corners_ra = []
    corners_dec = []
    for i in range(4):
        radec = wcs.pixelToSky(corners_x[i], corners_y[i])
        corners_ra.append(radec.getRa().asDegrees())
        corners_dec.append(radec.getDec().asDegrees())
    
    return corners_ra, corners_dec


In [ ]:
for datasetType in registry.queryDatasetTypes():
    if registry.queryDatasets(datasetType, collections=collections).any(execute=False, exact=False):
        # Limit search results to the data products
        if ('_config' not in datasetType.name) and ('_log' not in datasetType.name) and ('_metadata' not in datasetType.name) and ('_resource_usage' not in datasetType.name):
            print(datasetType)

## How do I figure out which tracts have data?

In [ ]:
for dtype in sorted(registry.queryDatasetTypes(expression="*nImage*")):
    print(dtype.name)

In [ ]:
nImage_refs = list(butler.registry.queryDatasets('deepCoadd_nImage'))

In [ ]:
tracts = np.unique([ref.dataId['tract'] for ref in nImage_refs])
print(tracts)

bands = np.unique([ref.dataId['band'] for ref in nImage_refs])
print(bands)

In [ ]:
tract_sel = tracts[22]
#tract_sel = 3533

In [ ]:
skymap = butler.get('skyMap', skymap=skymap_name)
tract = skymap.generateTract(tract_sel)
sp2 = tract.getCtrCoord()
sp2

In [ ]:
# Check which tracts actually have a lot of visit coverage:
for tract in tracts:
    visits = list(butler.registry.queryDatasets('visitSummary', tract=tract, skymap=skymap_name, findFirst=True))
    print(tract, len(visits))

In [ ]:
#datasetRefs = registry.queryDatasets(datasetType='deepCoadd', collections=collections, where= "instrument='LSSTComCamSim'")
datasetRefs = registry.queryDatasets(datasetType='deepCoadd', collections=collections, where = where_clause)
for i, ref in enumerate(datasetRefs):
    print(f"========({i})================datasetType = deepCoadd ============================================")
    #print("fullId..................:",ref.dataId.full)
    print("band....................:",ref.dataId["band"])
    #print("skymap..................:",ref.dataId["skymap"])
    print("tract..................:",ref.dataId["tract"])
    print("patch..................:",ref.dataId["patch"])
    #print("run.....................:",ref.run)
    if i> 5:
        break


### Select a tract and a band

In [ ]:
index_sel = 22
#index_sel = 7
#tract_sel = 9590
#tract_sel = 3533
tract_sel = tracts[index_sel]
band_sel = "r"

In [ ]:
tract_sel

In [ ]:
datasetRefs = registry.queryDatasets(datasetType='deepCoadd', collections=collections, where= "instrument='LSSTComCamSim'")
all_dataId = []
all_deepcoadds  = []
all_titles = []
all_patches = []
all_deepCoadd_wcs = []
all_deepCoadd_bbox = []
all_deepCoadd_corners_ra= []
all_deepCoadd_corners_dec = []
all_xvals = []
all_yvals = []
all_str_tract_patch = []

for i, ref in enumerate(datasetRefs):

    band = ref.dataId["band"]
    tract = ref.dataId["tract"]
    patch = ref.dataId["patch"]
    skymap = ref.dataId["skymap"]

    if band == band_sel and tract == tract_sel:
        print(f"========({i})================datasetType = deepCoadd ============================================")
        #print("fullId..................:",ref.dataId.full)
        #print("band....................:",ref.dataId["band"])
        #print("skymap..................:",ref.dataId["skymap"])
        #print("tract..................:",ref.dataId["tract"])
        #print("patch..................:",ref.dataId["patch"])
        #print("run.....................:",ref.run)
    
        
        the_dataId = {'skymap':skymap,'band': band, 'tract': tract, 'patch': patch}
        the_title = f"{instrument} deepCoadd : band={band}, tract={tract}, patch={patch}" 
        my_deepCoadd = butler.get('deepCoadd', dataId=ref.dataId.full,collections=collections)

        #my_deepCoadd_wcs = butler.get('deepCoadd.wcs', dataId=ref.dataId.full)
        #my_deepCoadd_bbox = butler.get('deepCoadd.bbox', dataId=ref.dataId.full)
        my_deepCoadd_wcs = my_deepCoadd.wcs
        my_deepCoadd_bbox = my_deepCoadd.getBBox() 
        my_deepCoadd_corners_ra, my_deepCoadd_corners_dec = get_corners_radec(my_deepCoadd_wcs, my_deepCoadd_bbox)
        my_xvals = [my_deepCoadd_corners_ra[0], my_deepCoadd_corners_ra[1], my_deepCoadd_corners_ra[2], \
             my_deepCoadd_corners_ra[3], my_deepCoadd_corners_ra[0]]
        my_yvals = [my_deepCoadd_corners_dec[0], my_deepCoadd_corners_dec[1], my_deepCoadd_corners_dec[2], \
             my_deepCoadd_corners_dec[3], my_deepCoadd_corners_dec[0]]

        str_tract_patch = '(' + str(ref.dataId['tract']) + ', ' + str(ref.dataId['patch'])+')'
        
        all_dataId.append(the_dataId) 
        all_patches.append(ref.dataId["patch"])
        all_deepcoadds.append(my_deepCoadd)
        all_titles.append(the_title)
        all_deepCoadd_wcs.append( my_deepCoadd_wcs)
        all_deepCoadd_bbox.append(my_deepCoadd_bbox)
        all_deepCoadd_corners_ra.append(my_deepCoadd_corners_ra)
        all_deepCoadd_corners_dec.append(my_deepCoadd_corners_dec)
        all_xvals.append(my_xvals)
        all_yvals.append(my_yvals)
        all_str_tract_patch.append(str_tract_patch)  
    else:
        pass
    


In [ ]:
N = len(all_deepcoadds)
for index in range(N):
    print(index,all_titles[index])

In [ ]:
all_patches = np.array(all_patches)
list_patches = np.unique(all_patches)

In [ ]:
all_dataId

In [ ]:
nPatches = len(all_patches)
sns.color_palette("hls", nPatches)
cmap = ListedColormap(sns.color_palette("hls", nPatches))

In [ ]:
len(all_patches)

In [ ]:
patch_to_color = {}
patch_flagplot = {}
for idx,patch in enumerate(all_patches):
    patch_to_color[patch] = cmap.colors[idx]
    patch_flagplot[patch] = False 


In [ ]:
fig = plt.figure(figsize=(8, 8))

for idx,dataid in enumerate(all_dataId):  
    patch = dataid['patch']
    if not patch_flagplot[patch]:
        xvals = np.array(all_xvals[idx])
        yvals = np.array(all_yvals[idx])
        xmean = xvals.mean()
        ymean = yvals.mean()
        
        str_tract_patch = all_str_tract_patch[idx]
        color = patch_to_color[patch]
        plt.plot(xvals, yvals, ls='solid', lw=2, color=color,label=str_tract_patch)
        patch_flagplot[patch] = True
        plt.text(xmean,ymean, str(patch),
                 verticalalignment='center', horizontalalignment='center', 
                 color=color,size=14)
plt.xlabel('RA')
plt.ylabel('Dec')
plt.legend(bbox_to_anchor=(1.1, 1.05), ncol=1)
the_title_tract = f"LSSTComCamSim : tract {tract_sel}"
plt.title(the_title_tract)
plt.gca().set_aspect('equal')
plt.show()

In [ ]:
N = len(all_deepcoadds)

for index in range(N):
    the_dataid = all_dataId[index]
    the_deepcoadd = all_deepcoadds[index]
    the_title = f"{index})  "+all_titles[index]

    xvals = np.array(all_xvals[index])
    yvals = np.array(all_yvals[index])

    extent=[xvals.min(),xvals.max(),yvals.min(),yvals.max()]
    
    # Plot the calexp we just retrieved
    plt.figure(figsize=(10,10))
    #afw_display = afwDisplay.Display()
    #afw_display.scale('asinh', 'zscale')
    #afw_display.mtv(the_deepcoadd.image,title=the_title )
    #plt.gca().axis('on')
    transform = AsinhStretch() + ZScaleInterval()
    scaledImage = transform(the_deepcoadd.image.array)
    #scaledImage = np.flipud(scaledImage)
    plt.imshow(scaledImage,origin="lower",extent=extent,aspect='equal')
    plt.title(the_title)
    plt.grid()
    #plt.gca().invert_xaxis()
    plt.show()
